In [26]:
n_channels = 32 
n_features = 5

In [56]:
import tensorflow as tf

from tensorflow.keras.layers import Conv2D, MaxPooling2D, concatenate, BatchNormalization, GlobalMaxPooling2D
from tensorflow.keras.layers import GlobalAveragePooling2D, Activation, Dropout, AveragePooling2D, Dense, Input
from tensorflow.keras.models import Model
from keras.regularizers import l2

Using TensorFlow backend.


In [57]:
def name_or_none(prefix, name):
    return prefix + name if (prefix is not None and name is not None) else None

In [168]:
def __conv_block(ip, nb_filter, dropout_rate=False, weight_decay=1e-4, block_prefix=None):
    with tf.name_scope('ConvBlock'):
        concat_axis = 1 if K.image_data_format() == 'channels_first' else -1
        
        x = BatchNormalization(axis=concat_axis, epsilon=1.1e-5,
                              name=name_or_none(block_prefix, '_bn'))(ip)
        
        x = Activation('relu')(x)
        
        x = Conv2D(nb_filter, (3,1), kernel_initializer='he_normal', padding='same',
                  use_bias=False, name=name_or_none(block_prefix, '_conv2D'))(x)
        
        if(dropout_rate):
            x = Dropout(dropout_rate)(x)
        
    return x

In [178]:
def __dense_block(x, nb_layers, nb_filter, growth_rate, dropout_rate, weight_decay, grow_nb_filters=True,
                 block_prefix=None):
    with tf.name_scope('DenseBlock'):
        concat_axis = 1 if K.image_data_format() == 'channels_first' else -1
        
        x_list = [x]
        
        for i in range(nb_layers):
            conv_block = __conv_block(x, growth_rate, dropout_rate, weight_decay,
                                     block_prefix = name_or_none(block_prefix, '_%i' % i))
            
            x_list.append(conv_block)
            
            x = concatenate([x, conv_block], axis = concat_axis)
            
            if grow_nb_filters:
                nb_filter += growth_rate
            
            
        return x, nb_filter

In [179]:
def __transition_block(ip, nb_filter, compression=1.0, weight_decay=1e-4,
                       block_prefix=None, transition_pooling='max'):
    with tf.name_scope('Transition'):
        concat_axis = 1 if K.image_data_format() == 'channels_first' else -1
        
        x = BatchNormalization(axis=concat_axis, epsilon=1.1e-5,
                               name=name_or_none(block_prefix, '_bn'))(ip)
        
        x = Activation('relu')(x)
        
        x = Conv2D(int(nb_filter * compression), (1,1), kernel_initializer='he_normal', padding='same',
                  use_bias=False, kernel_regularizer=l2(weight_decay), 
                  name=name_or_none(block_prefix,'_conv2D'))(x)
        
        if transition_pooling == 'avg':
            x = AveragePooling2D((2, 1), strides=(2, 2))(x)
        elif transition_pooling == 'max':
            x = MaxPooling2D((2, 1), strides=(2, 2))(x)
        
        return x

In [180]:
def __create_dense_net(nb_classes, img_input, depth=40, nb_dense_block=3,
                      growth_rate=12, nb_filter=-1, nb_layers_per_block=-1,
                      reduction=0.0, dropout_rate=None, weight_decay=1e-4, 
                      pooling=None, activation='softmax', transition_pooling='avg'):
    
    with tf.name_scope('Densenet'):
        concat_axis = 1 if K.image_data_format() == 'channels_first' else -1
        
        nb_layers = list(nb_layers_per_block)
        if len(nb_layers) != nb_dense_block:
            raise ValueError('If `nb_dense_block` is a list, its length must match '
                                 'the number of layers provided by `nb_layers`.')
        
        final_nb_layer = nb_layers[-1]
        nb_layers  = nb_layers[:-1]
            
        compression = 1.0 - reduction 
        
        initial_kernel = (1, 5)
        initial_strides = (1, 1)
        
        x = Conv2D(nb_filter, initial_kernel, kernel_initializer='he_normal',
                   padding='valid', name='initial_conv2D', strides=initial_strides,
                   use_bias=False, kernel_regularizer=l2(weight_decay))(img_input)
        
        x = BatchNormalization(axis=concat_axis, epsilon=1.1e-5,
                                   name='initial_bn')(x)
        x = Activation('relu')(x)
        
        for block_idx in range(nb_dense_block - 1):
            # Add dense blocks
            x, nb_filter = __dense_block(x, nb_layers[block_idx], nb_filter, 
                                         growth_rate, dropout_rate=dropout_rate, 
                                        weight_decay=weight_decay, 
                                        block_prefix='dense_%i' % block_idx)
            
            # Add trainsition block
            x = __transition_block(x, nb_filter, compression=compression,
                                   weight_decay=weight_decay,
                                   block_prefix='tr_%i' % block_idx)
            
            nb_filter = int(nb_filter * compression)
        
        # The last dense_block does not have a transition_block
        x, nb_filter = __dense_block(x, final_nb_layer, nb_filter, 
                                        growth_rate, dropout_rate=dropout_rate, 
                                        weight_decay=weight_decay, 
                                        block_prefix='dense_%i' % (nb_dense_block - 1))
        
        
        if pooling == 'avg':
            x = GlobalAveragePooling2D()(x) 
        elif pooling == 'max':
            x = GlobalMaxPooling2D()(x)
            
        x = Dense(nb_classes, activation=activation)(x)

        return x
    

In [181]:
def DenseNet(input_shape=None,
             depth=40,
             nb_dense_block=3,
             growth_rate=12,
             nb_filter=-1,
             nb_layers_per_block=-1,
             reduction=0.0,
             dropout_rate=0.0,
             weight_decay=1e-4,
             weights=None,
             input_tensor=None,
             pooling=None,
             classes=10,
             activation='softmax',
             transition_pooling='avg'):
    
    img_input = Input(shape=input_shape)
    
    x = __create_dense_net(classes, img_input, depth, nb_dense_block,
                           growth_rate, nb_filter, nb_layers_per_block,
                           reduction, dropout_rate, weight_decay,
                           pooling, activation, transition_pooling)
    
    # Create model.
    model = Model(img_input, x, name='densenet')
    
    return model

In [182]:
import tensorflow.keras.backend as K

In [183]:
K.set_image_data_format("channels_last")

In [184]:
K.image_data_format()

'channels_last'

In [187]:
m = DenseNet(input_shape=(n_channels, n_features,1),
             depth=40,
             nb_dense_block=3,
             growth_rate=12,
             nb_filter=24,
             nb_layers_per_block=[6,6,6],
             reduction=0.0,
             dropout_rate=0.0,
             weight_decay=1e-4,
             weights=None,
             input_tensor=None,
             pooling='avg',
             classes=10,
             activation='softmax',
             transition_pooling='avg')

In [188]:
m.summary()

Model: "densenet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_20 (InputLayer)           [(None, 32, 5, 1)]   0                                            
__________________________________________________________________________________________________
initial_conv2D (Conv2D)         (None, 32, 1, 24)    120         input_20[0][0]                   
__________________________________________________________________________________________________
initial_bn (BatchNormalization) (None, 32, 1, 24)    96          initial_conv2D[0][0]             
__________________________________________________________________________________________________
activation_61 (Activation)      (None, 32, 1, 24)    0           initial_bn[0][0]                 
___________________________________________________________________________________________